<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# Kafka Consumer 

**Technical Accomplishments:**
- Introduce the class `Consumer`
- Consume data from Kafka topic

## Getting Started

Let's start importing libraries and creating useful variables 

In [ ]:
%load_ext autotime

In [ ]:
from confluent_kafka import Consumer, KafkaError
import json
import qcutils

topic = ''
consumer_group = ''

assert len(topic) > 0, "In order to avoid conflicts during write operation, please name the topic as <surname>-topic"
assert len(consumer_group) > 0, "In order to avoid conflicts during write operation, please name the consumer_group as <surname>-cg"


servers=qcutils.read_config_value("kafka.server") + ":" + str(qcutils.read_config_value("kafka.port"))

**Note**: in order to avoid conflicts during read operation, please name:
* the topic as `<surname>-topic`
* the consumer group as follow `<surname>-cg`

In [ ]:
consumerconf = {
        'bootstrap.servers': servers,
        'group.id': consumer_group,
        'auto.offset.reset': 'earliest'
    }

c = Consumer(consumerconf)

## JSON Consumer

With the `json.loads(msg.value())` we directly load the value into a json object, that can be navigated.

In [ ]:
c.subscribe([topic])

waiting = False

try:
    while True:
        msg = c.poll(1.0)                                        # retrieve records
        if msg is None:
            if waiting:
                print(".",end =" ")
            else:
                print("Waiting",end =" ")
                waiting = True
            continue
        elif msg.error():
            print('error: {}'.format(msg.error()))
            waiting = False
        else:
            value = json.loads(msg.value())                      # get the value of the message (json.loads)
            key = msg.key()                                      # get the key of the message
            print("\nConsumed record with key {} and value {}"
                  .format(key, value))
            waiting = False
except KeyboardInterrupt:
    pass
finally:
    c.close()                                                   # Leave group and commit final offsets

**Note** In order to add SASL security for the connection to the kafka broker, you need to add security configurations.

```
username=qcutils.read_config_value("kafka.access.key")
password=qcutils.read_config_value("kafka.access.secret")

consumerconf = {
        'bootstrap.servers': <servers>,
        'sasl.mechanisms': 'PLAIN',
        'security.protocol': 'SASL_SSL',
        'sasl.username': <username>,
        'sasl.password': <password>,
        'group.id': <consumer-group>,
        'auto.offset.reset': <....>
    }
```

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2020 Quantia Consulting, srl. All rights reserved.